In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from matplotlib import pyplot as plt
import yaml
import numpy as np

from grconvnet.dataloading.datasets import YCBSimulationData
from grconvnet.utils.processing import End2EndProcessor
from grconvnet.postprocessing import Img2WorldCoordConverter, Decropper
from grconvnet.utils.config import module_from_config
from grconvnet.utils import visualization as vis
from grconvnet.utils.export import Exporter

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataset_path = Path("/home/moritz/Documents/ycb_sim_data_1")
config_path = Path.cwd().parent / "grconvnet" / "configs" / "ycb_inference_standard.yaml"
export_path = Path.cwd().parent / "grconvnet" / "results" / "ycb_1"

In [3]:
dataset = YCBSimulationData(dataset_path)
sample = dataset[0]
sample.name

'001_master_chef_can'

In [4]:
cam_intrinsics = sample.cam_intrinsics
cam_pos = sample.cam_pos
cam_rot = sample.cam_rot
image_size = sample.rgb.shape[1:]

dataset_config = {
    "path": str(dataset_path),
    "cam_intrinsics": cam_intrinsics.tolist(),
    "cam_pos": cam_pos.tolist(),
    "cam_rot": cam_rot.tolist(),
    "image_size": list(image_size), 
}

In [5]:
with open(config_path) as f:
    config = yaml.safe_load(f)

In [6]:
e2e_processor = module_from_config(config)

e2e_processor.img2world_converter.coord_converter = Img2WorldCoordConverter(
    cam_intrinsics, cam_rot, cam_pos
)
e2e_processor.img2world_converter.decropper = Decropper(
    resized_in_preprocess=config["preprocessor"]["resize"], original_img_size=image_size
)

In [7]:
exporter = Exporter(export_dir=export_path)

export_path.mkdir(parents=True, exist_ok=True)

with open(export_path / "inference_config.yaml", "w") as f:
    yaml.dump(config, f)

with open(export_path / "dataset_config.yaml", "w") as f:
    yaml.dump(dataset_config, f)

In [8]:
for sample in dataset:
    print(f"Processing sample {sample.name}...")

    assert np.allclose(sample.cam_intrinsics, cam_intrinsics)
    assert np.allclose(sample.cam_pos, cam_pos)
    assert np.allclose(sample.cam_rot, cam_rot)
    assert np.allclose(sample.rgb.shape[1:], image_size)

    process_data = e2e_processor(sample)

    fig = vis.overview_fig(
        fig=plt.figure(figsize=(20, 20)),
        original_rgb=vis.make_tensor_displayable(process_data["sample"].rgb, True, True),
        preprocessed_rgb=vis.make_tensor_displayable(
            process_data["preprocessor"]["rgb_masked"], True, True
        ),
        q_img=vis.make_tensor_displayable(
            process_data["postprocessor"]["q_img"], False, False
        ),
        angle_img=vis.make_tensor_displayable(
            process_data["postprocessor"]["angle_img"], False, False
        ),
        width_img=vis.make_tensor_displayable(
            process_data["postprocessor"]["width_img"], False, False
        ),
        image_grasps=process_data["grasps_img"],
        world_grasps=process_data["grasps_world"],
        cam_intrinsics=cam_intrinsics,
        cam_rot=cam_rot,
        cam_pos=cam_pos,
    )

    export_data = {
        "rgb_cropped": process_data["preprocessor"]["rgb_cropped"],
        "depth_cropped": process_data["preprocessor"]["depth_cropped"],
        "rgb_masked": process_data["preprocessor"]["rgb_masked"],
        "q_img": process_data["postprocessor"]["q_img"],
        "angle_img": process_data["postprocessor"]["angle_img"],
        "width_img": process_data["postprocessor"]["width_img"],
        "grasps_img": process_data["grasps_img"],
        "grasps_world": process_data["grasps_world"],
        "model_input": process_data["model_input"],
        "overview": fig,
    }

    export_path = exporter(export_data, f"{process_data['sample'].name}")

Processing sample 001_master_chef_can...
Processing sample 002_cracker_box...
Processing sample 003_sugar_box...
Processing sample 004_tomato_soup_can...
Processing sample 005_mustard_bottle...
Processing sample 006_tuna_fish_can...
Processing sample 007_pudding_box...
Processing sample 008_gelatin_box...
Processing sample 009_potted_meat_can...
Processing sample 010_banana...
Processing sample 011_strawberry...
Processing sample 012_apple...
Processing sample 013_lemon...
Processing sample 014_peach...
Processing sample 015_pear...
Processing sample 016_orange...
Processing sample 017_plum...
Processing sample 018_pitcher_base...
Processing sample 019_bleach_cleanser...
Processing sample 021_bowl...
Processing sample 022_mug...


/tmp/ipykernel_9653/334740947.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig=plt.figure(figsize=(20, 20)),


Processing sample 023_sponge...
Processing sample 025_plate...
Processing sample 026_fork...
Processing sample 027_spoon...
Processing sample 028_knife...
Processing sample 029_spatula...
Processing sample 030_power_drill...
Processing sample 031_wood_block...
Processing sample 032_scissors...
Processing sample 034_large_marker...
Processing sample 036_adjustable_wrench...
Processing sample 037_phillips_screwdriver...
Processing sample 038_flat_screwdriver...
Processing sample 039_hammer...
Processing sample 041_medium_clamp...
Processing sample 042_large_clamp...
Processing sample 043_extra_large_clamp...
Processing sample 044_mini_soccer_ball...
Processing sample 045_softball...
Processing sample 046_baseball...
Processing sample 047_tennis_ball...
Processing sample 048_racquetball...
Processing sample 049_golf_ball...
Processing sample 050_chain...
Processing sample 051_foam_brick...
Processing sample 052_dice...
Processing sample 053_a_marbles...
Processing sample 054_b_marbles...
